In [750]:
import requests
from bs4 import BeautifulSoup
from nltk.tokenize import TweetTokenizer
import re
import nltk
import fractions

In [700]:
get_ingredients('https://www.allrecipes.com/recipe/24074/alysias-basic-meat-lasagna/')

{'ground beef ': ['1 1/2', 'pounds', [], []],
 'powder ': ['1', 'teaspoon', ['garlic'], []],
 'sausage spaghetti ': ['1', 'jar', [], []],
 'tomato sauce ': ['1', 'can', [], []],
 'oregano ': ['teaspoon', 'teaspoon', [], ['dried']],
 'oil ': ['1', 'tablespoon', ['olive'], []],
 'garlic ': ['4', 'cloves', [], [' minced']],
 'onion ': ['1', 'None', ['small'], [' diced']],
 'mozzarella cheese ': ['1', 'package', [], [' shredded']],
 'provolone cheese ': ['8', 'ounces', [], [' shredded']],
 'ricotta cheese ': ['1', 'container', [], []],
 'eggs ': ['2', 'None', [], []],
 'milk ': ['cup', 'cup', [], []],
 'lasagna noodles ': ['9', 'None', [], []],
 'cheese ': ['cup', 'cup', ['parmesan'], ['grated']]}

In [734]:
#input is a url as a string
def get_ingredients(url):
    ingredients = scrape_ingredients(url)
    ingreds = {}
    for i in ingredients:
        tknzr = TweetTokenizer()
        t = tknzr.tokenize(i)
        if '/' in t[1]:
            quantity = t[0] + ' ' + t[1]
        
        elif not hasNumbers(t[0]):
            quantity = t[1]
        else:
            quantity = t[0]
        
        measurement = get_measurement(i)

        name = get_name(i)
        pos = nltk.pos_tag(name.split())
        nom = []
        for k in pos:
            if k[1] == 'NNS':
                nom.append(k[0])
            if k[1] == 'NN':
                nom.append(k[0])
            if k[1] == 'garlic':
                nom.append(k[0])
        n = ''
        for j in nom:
            n = n + j + ' '

        pos = get_descriptor(name)

        prep = get_prep(name, i)

        ingreds[str(n)] = [str(quantity), str(measurement), pos, prep]
        
    #print('{name : [quantity, measurement, descriptor, preparation]}') 
    return ingreds
        
    

In [733]:
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)

In [702]:
def scrape_ingredients(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    ingredients_html = soup.findAll(class_ = 'checkList__line')
    ingredients = []
    for i in ingredients_html:
        try:
            mystr = str(i)
            ingredients.append(mystr.split('title=')[1].split('>')[0].strip('"'))
        except:
            pass
        
    return ingredients
     
    

In [703]:
def get_measurement(ingred):
    units = ['package','slice', 'container','litre', 'gallon', 'quart', 'cup', 'pint', 'jar', 'can', 'clove', 'ounce', 'spoon', 'gram', 'pound', 'pinch']
    tknzr = TweetTokenizer()
    t = tknzr.tokenize(ingred)
    for u in units:
        for w in t:
            if u in w:
                return w
        
    

In [704]:
def get_name(ingred):
    tknzr = TweetTokenizer()
    t = tknzr.tokenize(ingred)
    m = get_measurement(ingred)
    try:
        n = re.search(r'\d+', ingred).group()
        if (m and m != 'Not found' and m != 'None'):
            rest = ingred.split(m)[1]
            if ',' in rest:
                return rest.split(',')[0].lower()
            else:
                return rest.lower()
        else:
            rest = ingred.split(n)[1]
            if ',' in rest:
                return rest.split(',')[0].lower()
            else:
                return rest
    except:
        return ingred
        

In [705]:
def get_descriptor(name):
    descrip = []
    pos = nltk.pos_tag(name.split())
    for i in pos:
        if i[1] == 'JJ':
            descrip.append(i[0])
    return descrip

In [706]:
def get_prep(name, ingred):
    prep = []
    pos = nltk.pos_tag(name.split())
    for i in pos:
        if i[1] == 'VBN':
            prep.append(i[0])

    tknzr = TweetTokenizer()
    t = tknzr.tokenize(ingred)

    m = get_measurement(ingred)
    
    try:
        n = re.search(r'\d+', ingred).group()


        if (m and m != 'Not found' and m != 'None'):
            rest = ingred.split(m)[1]
            if ',' in rest:
                prep.append(rest.split(',')[1].lower())
        else:
            rest = ingred.split(n)[1]
            if ',' in rest:
                prep.append(rest.split(',')[1].lower())
        return prep
    except:
        return ''

# testing

In [707]:
url = 'https://www.allrecipes.com/recipe/218091/classic-and-simple-meat-lasagna/'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')
ingredients_html = soup.findAll(class_ = 'checkList__line')

In [708]:
ingredients = []
for i in ingredients_html:
    try:
        mystr = str(i)
        ingredients.append(mystr.split('title=')[1].split('>')[0].strip('"'))
    except:
        pass
     

In [672]:
ingreds = {}

In [673]:
ingredients

['12 whole wheat lasagna noodles',
 '1 pound lean ground beef',
 '2 cloves garlic, chopped',
 '1/2 teaspoon garlic powder',
 '1 teaspoon dried oregano, or to taste',
 'salt and ground black pepper to taste',
 '1 (16 ounce) package cottage cheese',
 '2 eggs',
 '1/2 cup shredded Parmesan cheese',
 '1 1/2 (25 ounce) jars tomato-basil pasta sauce',
 '2 cups shredded mozzarella cheese']

In [678]:
for i in ingredients:
    tknzr = TweetTokenizer()
    t = tknzr.tokenize(i)
    if '/' in t[1]:
        quantity = t[0] + ' ' + t[1]
    elif not t[0].isnumeric():
        quantity = t[1]
    else:
        quantity = t[0]
    
    measurement = get_measurement(i)
    
    name = get_name(i)
    pos = nltk.pos_tag(name.split())
    nom = []
    for k in pos:
        if k[1] == 'NNS':
            nom.append(k[0])
        if k[1] == 'NN':
            nom.append(k[0])
        if k[1] == 'garlic':
            nom.append(k[0])
    n = ''
    for j in nom:
        n = n + j + ' '
        
    pos = get_descriptor(name)
    
    prep = get_prep(name, i)
    
    ingreds[str(n)] = [str(quantity), str(measurement), pos, prep]

In [679]:
print('{name : [quantity, measurement, descriptor, preparation]}')
ingreds

{name : [quantity, measurement, descriptor, preparation]}


{'wheat lasagna noodles ': ['12', 'None', ['whole'], []],
 'ground beef ': ['1', 'pound', ['lean'], []],
 'garlic ': ['2', 'cloves', [], [' chopped']],
 'powder ': ['teaspoon', 'teaspoon', ['garlic'], []],
 'oregano ': ['1', 'teaspoon', [], ['dried', ' or to taste']],
 'salt ground pepper ': ['and', 'None', ['black'], ''],
 'cottage cheese ': ['1', 'package', [], []],
 'eggs ': ['2', 'None', [], []],
 'cheese ': ['cup', 'cup', ['parmesan'], ['shredded']],
 'pasta sauce ': ['1 1/2', 'jars', ['tomato-basil'], []],
 'mozzarella cheese ': ['2', 'cups', [], ['shredded']]}

# make_veg, make_meat, double

In [746]:
def make_veg(i_dict):
    new_dict = {}
    meat = ['beef', 'chicken', 'pork', 'salmon', 'tuna', 'sausage', 'veal', 'turkey', 'duck', 'goat', 'lamb', 'meatball', 
            'shrimp', 'crab', 'lobster', 'hot dog', 'bacon', 'ham', 'burger', 'patty', 'pork']
     
    for key, value in i_dict.items():
        new_value = [value[0], value[1], [], []]
        start = len(new_dict)
        i = 0
        for m in meat:
            if m in key:
                i = 1
                new_key = key.replace(m, 'tofu')
                new_dict[new_key] = new_value
                
        if len(new_dict) == start:
            if i == 1:
                new_key = key.replace(m, 'seitan')
                new_dict[new_key] = new_value
            else:
                new_dict[key] = value
    return new_dict


In [748]:
make_veg(get_ingredients('https://www.allrecipes.com/recipe/269230/sheet-pan-smoked-sausage-apple-and-root-veggie-dinner/?internalSource=rotd&referringContentType=Homepage'))

{'baby carrots ': ['8', 'ounces', [], []],
 'brussels sprouts ': ['8', 'ounces', [], [' halved lengthwise']],
 'onion ': ['1', 'None', ['small', 'red'], [' peeled and cut into 8 wedges']],
 'oil ': ['3', 'tablespoons', ['olive'], []],
 'rosemary ': ['2', 'teaspoons', ['fresh'], ['chopped']],
 'kosher salt ': ['1', 'teaspoon', [], []],
 'pepper ': ['1/2', 'teaspoon', ['black'], []],
 'hillshire farm(r) rope tofu ': ['1', 'package', [], []],
 'medium honeycrisp apple ': ['1', 'None', [], [' cut into ']],
 'lemon ': ['1', 'None', [], [' halved']],
 'parsley ': ['2', 'tablespoons', [], ['chopped']]}

In [764]:
def double_amount(i_dict):
    for key, value in i_dict.items():
        if '/' in value[0]:
            fraction_obj = sum(map(fractions.Fraction, value[0].split()))
            doubled = fraction_obj + fraction_obj
            n = doubled.numerator 
            d = doubled.denominator 
            if str(n) == str(d):
                amount = '1'
            else:
                amount = str(n) + str('/') + str(d)
            i_dict[key] = [amount, value[1], value [2], value[3]] 
        elif value[0].isnumeric():
            amount = int(value[0]) * 2
            i_dict[key] = [amount, value[1], value [2], value[3]]
        else:
            i_dict[key] = value
    return i_dict
            

In [766]:
double_amount(get_ingredients('https://www.allrecipes.com/recipe/269230/sheet-pan-smoked-sausage-apple-and-root-veggie-dinner/?internalSource=rotd&referringContentType=Homepage'))

{'baby carrots ': [16, 'ounces', [], []],
 'brussels sprouts ': [16, 'ounces', [], [' halved lengthwise']],
 'onion ': [2, 'None', ['small', 'red'], [' peeled and cut into 8 wedges']],
 'oil ': [6, 'tablespoons', ['olive'], []],
 'rosemary ': [4, 'teaspoons', ['fresh'], ['chopped']],
 'kosher salt ': [2, 'teaspoon', [], []],
 'pepper ': ['1', 'teaspoon', ['black'], []],
 'hillshire farm(r) rope sausage ': [2,
  'package',
  [],
  [' cut in 1/2" bias-cut slices\'']],
 'medium honeycrisp apple ': [2, 'None', [], [' cut into ']],
 'lemon ': [2, 'None', [], [' halved']],
 'parsley ': [4, 'tablespoons', [], ['chopped']]}

In [783]:
def make_meat(i_dict):
    new_dict = {}
    subs = ['tofu', 'eggplant', 'seitan', 'jackfruit']
    i = 0
    for key, value in i_dict.items():
        new_value = [value[0], 'pound(s)', [], []]
        start = len(new_dict)
        
        for m in subs:
            if m in key:
                i = i + 1
                new_key = key.replace(m, 'chicken')
                new_dict[new_key] = new_value
                
        if len(new_dict) == start:
                new_dict[key] = value
    if i == 0:
        new_dict['chicken'] = [1, 'pound', [], []]
    
    return new_dict


In [784]:
test_dict = {'baby carrots': [16, 'ounces', [], []],
 'brussels sprouts ': [16, 'ounces', [], [' halved lengthwise']],
 'onion ': [2, 'None', ['small', 'red'], [' peeled and cut into 8 wedges']],
 'oil ': [6, 'tablespoons', ['olive'], []],
 'rosemary ': [4, 'teaspoons', ['fresh'], ['chopped']],
 'kosher salt ': [2, 'teaspoon', [], []],
 'pepper ': ['1', 'teaspoon', ['black'], []],
 'eggplant ': [2,
  'packages',
  [],
  []],
 'medium honeycrisp apple ': [2, 'None', [], [' cut into ']],
 'lemon ': [2, 'None', [], [' halved']],
 'parsley ': [4, 'tablespoons', [], ['chopped']]}

In [785]:
test_dict1 = {'baby carrots': [16, 'ounces', [], []],
 'brussels sprouts ': [16, 'ounces', [], [' halved lengthwise']],
 'onion ': [2, 'None', ['small', 'red'], [' peeled and cut into 8 wedges']],
 'oil ': [6, 'tablespoons', ['olive'], []],
 'rosemary ': [4, 'teaspoons', ['fresh'], ['chopped']],
 'kosher salt ': [2, 'teaspoon', [], []],
 'pepper ': ['1', 'teaspoon', ['black'], []],
 'medium honeycrisp apple ': [2, 'None', [], [' cut into ']],
 'lemon ': [2, 'None', [], [' halved']],
 'parsley ': [4, 'tablespoons', [], ['chopped']]}

In [787]:
make_meat(test_dict1)

{'baby carrots': [16, 'ounces', [], []],
 'brussels sprouts ': [16, 'ounces', [], [' halved lengthwise']],
 'onion ': [2, 'None', ['small', 'red'], [' peeled and cut into 8 wedges']],
 'oil ': [6, 'tablespoons', ['olive'], []],
 'rosemary ': [4, 'teaspoons', ['fresh'], ['chopped']],
 'kosher salt ': [2, 'teaspoon', [], []],
 'pepper ': ['1', 'teaspoon', ['black'], []],
 'medium honeycrisp apple ': [2, 'None', [], [' cut into ']],
 'lemon ': [2, 'None', [], [' halved']],
 'parsley ': [4, 'tablespoons', [], ['chopped']],
 'chicken': [1, 'pound', [], []]}